In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections;
import re;
from time import time
import math
import feather
from collections import defaultdict

In [2]:
# Set up Socrata
client = Socrata("data.sfgov.org", None)
socrata_id_older = "cuks-n6tp"
socrata_id_newer = "wg3w-h783"

In [3]:
# Get Metadata for 2003-2018
metadata_old = client.get_metadata(socrata_id_older)
[x['name'] for x in metadata_old['columns']]

[u'IncidntNum',
 u'Category',
 u'Descript',
 u'DayOfWeek',
 u'Date',
 u'Time',
 u'PdDistrict',
 u'Resolution',
 u'Address',
 u'X',
 u'Y',
 u'Location',
 u'Location (city)',
 u'Location (address)',
 u'Location (zip)',
 u'Location (state)',
 u'PdId',
 u'Fire Prevention Districts',
 u'Police Districts',
 u'Supervisor Districts',
 u'Zip Codes',
 u'Neighborhoods',
 u'2017 Fix It Zones',
 u'Civic Center Harm Reduction Project Boundary',
 u'Neighborhoods - Analysis Boundaries',
 u'Fix It Zones as of 2017-11-06 ',
 u'HSOC Zones',
 u'Fix It Zones as of 2017-02-07',
 u'CBD, BID and GBD Boundaries as of 2017',
 u'Areas of Vulnerability, 2016',
 u'Central Market/Tenderloin Boundary',
 u'Central Market/Tenderloin Boundary Polygon - Updated',
 u'HSOC Zones as of 2018-06-05',
 u'Current Supervisor Districts',
 u'OWED Public Spaces',
 u'Current Police Districts']

In [4]:
# Get Metadata for 2018-2019
metadata_new = client.get_metadata(socrata_id_newer)
[x['name'] for x in metadata_new['columns']]

[u'Incident Datetime',
 u'Incident Date',
 u'Incident Time',
 u'Incident Year',
 u'Incident Day of Week',
 u'Report Datetime',
 u'Row ID',
 u'Incident ID',
 u'Incident Number',
 u'CAD Number',
 u'Report Type Code',
 u'Report Type Description',
 u'Filed Online',
 u'Incident Code',
 u'Incident Category',
 u'Incident Subcategory',
 u'Incident Description',
 u'Resolution',
 u'Intersection',
 u'CNN',
 u'Police District',
 u'Analysis Neighborhood',
 u'Supervisor District',
 u'Latitude',
 u'Longitude',
 u'point']

In [5]:
col_new = []
col_old = []

for i in range(len(metadata_new['columns'])):
    name = metadata_new['columns'][i]['name']
    names = []
    names.append(name)
    col_new = col_new + names

for i in range(len(metadata_old['columns'])):
    name = metadata_old['columns'][i]['name']
    names = []
    names.append(name)
    col_old = col_old + names

In [6]:
# Import 2018-2019 data via Socrata

loop_size = 10000
num_loops = 20

df_new = pd.DataFrame(columns=col_new)

for i in range(num_loops):
    results = client.get(socrata_id_newer,
                         limit=loop_size,
                         offset=loop_size * i)
    print("\n> Loop number: {}".format(i))
    df_loop = pd.DataFrame(results)
    df_new = pd.concat([df_new,df_loop])


> Loop number: 0

> Loop number: 1

> Loop number: 2

> Loop number: 3

> Loop number: 4

> Loop number: 5

> Loop number: 6

> Loop number: 7

> Loop number: 8

> Loop number: 9

> Loop number: 10

> Loop number: 11

> Loop number: 12

> Loop number: 13

> Loop number: 14

> Loop number: 15

> Loop number: 16

> Loop number: 17

> Loop number: 18

> Loop number: 19


In [7]:
# Import 2003-2018 data via Socrata 
loop_size = 25000
num_loops = 21

df_old = pd.DataFrame(columns=col_old)

for i in range(num_loops):
    results = client.get(socrata_id_older,
                         where="date_extract_y(Date) > 2014",
                         limit=loop_size,
                         offset=loop_size * i)
    print("\n> Loop number: {}".format(i))
    df_loop = pd.DataFrame(results)
    df_old = pd.concat([df_old,df_loop])


> Loop number: 0

> Loop number: 1

> Loop number: 2

> Loop number: 3

> Loop number: 4

> Loop number: 5

> Loop number: 6

> Loop number: 7

> Loop number: 8

> Loop number: 9

> Loop number: 10

> Loop number: 11

> Loop number: 12

> Loop number: 13

> Loop number: 14

> Loop number: 15

> Loop number: 16

> Loop number: 17

> Loop number: 18

> Loop number: 19

> Loop number: 20


In [8]:
print("Data - 2018 to 2019")
print(df_new.shape)
print(df_new.columns)

print("Data - 2014 to 2018")
print(df_old.shape)
print(df_old.columns)

Data - 2018 to 2019
(193082, 51)
Index([u'Analysis Neighborhood', u'CAD Number', u'CNN', u'Filed Online',
       u'Incident Category', u'Incident Code', u'Incident Date',
       u'Incident Datetime', u'Incident Day of Week', u'Incident Description',
       u'Incident ID', u'Incident Number', u'Incident Subcategory',
       u'Incident Time', u'Incident Year', u'Intersection', u'Latitude',
       u'Longitude', u'Police District', u'Report Datetime',
       u'Report Type Code', u'Report Type Description', u'Resolution',
       u'Row ID', u'Supervisor District', u'analysis_neighborhood',
       u'cad_number', u'cnn', u'filed_online', u'incident_category',
       u'incident_code', u'incident_date', u'incident_datetime',
       u'incident_day_of_week', u'incident_description', u'incident_id',
       u'incident_number', u'incident_subcategory', u'incident_time',
       u'incident_year', u'intersection', u'latitude', u'longitude', u'point',
       u'police_district', u'report_datetime', u'repo

In [9]:
## Data cleaning - keep variables in df_new

df_new = df_new[['incident_category', 'incident_date', 
                 'incident_day_of_week', 'incident_description', 'incident_number',
                 'incident_time', 'latitude', 'longitude', 'police_district']]

df_new = df_new.rename(columns={'incident_day_of_week': 'incident_dow'})


In [10]:
print(df_old.columns)

Index([u'2017 Fix It Zones', u'Address', u'Areas of Vulnerability, 2016',
       u'CBD, BID and GBD Boundaries as of 2017', u'Category',
       u'Central Market/Tenderloin Boundary',
       u'Central Market/Tenderloin Boundary Polygon - Updated',
       u'Civic Center Harm Reduction Project Boundary',
       u'Current Police Districts', u'Current Supervisor Districts', u'Date',
       u'DayOfWeek', u'Descript', u'Fire Prevention Districts',
       u'Fix It Zones as of 2017-02-07', u'Fix It Zones as of 2017-11-06 ',
       u'HSOC Zones', u'HSOC Zones as of 2018-06-05', u'IncidntNum',
       u'Location', u'Location (address)', u'Location (city)',
       u'Location (state)', u'Location (zip)', u'Neighborhoods',
       u'Neighborhoods - Analysis Boundaries', u'OWED Public Spaces',
       u'PdDistrict', u'PdId', u'Police Districts', u'Resolution',
       u'Supervisor Districts', u'Time', u'X', u'Y', u'Zip Codes', u'address',
       u'category', u'date', u'dayofweek', u'descript', u'incidntn

In [11]:
## Data cleaning - keep variables in df_old

df_old = df_old[['category', 'date', 'dayofweek', 'descript', 'incidntnum', 'time', 'x', 'y', 'pddistrict']]

df_old = df_old.rename(columns={
        'category': 'incident_category',
        'date': 'incident_date',
        'dayofweek': 'incident_dow',
        'descript':'incident_description',
        'incidntnum': 'incident_number',
        'time': 'incident_time',
        'x': 'latitude',
        'y': 'longitude',
        'pddistrict': 'police_district'})



In [12]:
df_old.incident_date.head()

0    2017-10-17T00:00:00.000
1    2016-09-01T00:00:00.000
2    2017-11-14T00:00:00.000
3    2018-04-07T00:00:00.000
4    2018-02-15T00:00:00.000
Name: incident_date, dtype: object

In [13]:
## Format Dates, dealing with different formats
# 1) create new column temp_date and store there datetimes recognized using first format
df_new['temp_date'] = pd.to_datetime(df_new['incident_date'], format='%Y/%m/%d', errors='coerce')
# 2) get not recognized rows
not_recongized = df_new[df_new.temp_date.isnull()]
# 3) inplace fill not recognized rows with values recognized using second format
df_new['temp_date'].fillna(pd.to_datetime(not_recongized['incident_date'], format='%Y/%m/%d', errors='coerce'), inplace=True)

# 4) delete starttime column
del df_new['incident_date']
# 5) inplace rename temp_starttime to starttime
df_new.rename(columns={"temp_date": "incident_date"}, inplace=True)

df_new['incident_date'] = pd.to_datetime(df_new['incident_date'],format='%m/%d/%Y') 

df_old['incident_date'] = df_old.incident_date.str.slice(0, 10)
df_old['incident_date'] = pd.to_datetime(df_old['incident_date'],format='%Y/%m/%d') 


In [14]:
## Merge incident data
print(len(df_old))
print(len(df_new))
df_incident = pd.concat([df_new, df_old])

print(df_incident.shape)
print(df_incident.head(5))


508850
193082
(701932, 9)
     incident_category incident_date  \
0             Homicide    2018-08-20   
1              Assault    2018-08-11   
2  Other Miscellaneous    2018-08-14   
3        Larceny Theft    2018-08-14   
4  Other Miscellaneous    2018-08-14   

                                incident_description incident_dow  \
0                          Homicide, W/ Other Weapon       Monday   
1               Assault, Aggravated, W/ Other Weapon     Saturday   
2  Resisting, Delaying, or Obstructing  Peace Off...      Tuesday   
3                              Theft, Bicycle, >$950      Tuesday   
4                      Burglary Tools, Possession Of      Tuesday   

  incident_number incident_time            latitude            longitude  \
0       180624631         07:45   37.78610103879033  -122.41808809883916   
1       180602295         19:10   37.75683373380551  -122.40669900268833   
2       189002400         14:15  37.770555975146884  -122.39115601762397   
3       180609

In [15]:
feather.write_dataframe(df_incident, 'data/sf_incidents.feather')